<a href="https://colab.research.google.com/github/PhoenixChheav/Medical-Image-Classificatiom/blob/main/DatasetAndViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install medmnist

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.5 MB/s eta 0:00:00


In [3]:
import medmnist
from medmnist import INFO
from medmnist.dataset import MedMNIST
from torchvision import transforms
from torch.utils.data import DataLoader